<a href="https://colab.research.google.com/github/LeonVillanueva/CoLab/blob/master/Google_CoLab_Simple_Bidirectional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow==2.0.0-beta-1

     |████████████████████████████████| 87.9MB 47kB/s 
     |████████████████████████████████| 501kB 57.7MB/s 
     |████████████████████████████████| 3.1MB 18.1MB/s 


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [0]:
%%capture

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [0]:
from tensorflow.keras.layers import InputLayer, LSTM, GRU, Input, GlobalMaxPooling1D, Bidirectional
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.models import Model

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam
from sklearn.metrics import roc_auc_score

Using TensorFlow backend.


In [10]:
!unzip train.csv

Archive:  train.csv.zip
  inflating: train.csv               


In [0]:
data = pd.read_csv  ('train.csv')

In [0]:
text_column = data['comment_text']

In [13]:
np.max (text_column.apply(len)), data.shape

(1906, (1804874, 45))

In [14]:
from nltk.corpus import stopwords
import nltk
import re

nltk.download ('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
test_stop = stopwords.words ('english')

In [0]:
def clean_text(x):
    x = x.lower()
    x = re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", x)
    stop_words = stopwords.words ('english')
    words = re.sub(r'[^\w\s]','',x).split(' ')
    words = [word for word in words if word not in stop_words]
    return words

In [0]:
text = text_column.apply (clean_text)

In [18]:
np.max (text_column.apply(len))

1906

In [23]:
len (text_column[~(text_column.map(len) <= 100)]) # for speed removing only under 100 items

1318940

In [0]:
flags = ['severe_toxicity', 'obscene','identity_attack', 'insult', 'threat']

In [0]:
MAX_SEQ_LENGTH = np.max (text.apply(len))
MAX_VOCAB_SIZE = 30000
EMBEDDING_DIM = 50

In [0]:
text = text.values

In [0]:
targets = data[flags].values

In [0]:
X_train, X_test, y_train, y_test = train_test_split (text, targets, random_state=23, test_size=0.2)

In [0]:
X_train.shape, X_test.shape

In [0]:
tokenizer = Tokenizer (num_words = MAX_VOCAB_SIZE)

In [0]:
tokenizer.fit_on_texts (X_train)

In [0]:
X_train_seq = tokenizer.texts_to_sequences (X_train)
X_test_seq = tokenizer.texts_to_sequences (X_test)

In [0]:
X_train_seq_pad = pad_sequences (X_train_seq, maxlen=MAX_SEQ_LENGTH, padding='post')
X_test_seq_pad = pad_sequences (X_test_seq, maxlen=MAX_SEQ_LENGTH, padding='post')